# Prefix-Tuning 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset 
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

c:\Users\G\miniforge3\envs\transformers\lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[1]

{'output': '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
 'input': '输入：4/16',
 'instruction': '解释为什么以下分数等同于1/4'}

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("C:/Pretrained_model/LLM-Research/Llama-3___2-3B-Instruct")
tokenizer

PreTrainedTokenizerFast(name_or_path='C:/Pretrained_model/LLM-Research/Llama-3___2-3B-Instruct', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|rese

In [6]:
#tokenizer.added_tokens_decoder[128004]

# tokenizer.pad_token=tokenizer.added_tokens_decoder[128004]
tokenizer.pad_token_id = 128004

In [9]:
def process_func(example):
     # Step 1: Get the instruction, input, and output from the example
    instruction = example['instruction']  # Instruction field (e.g., '解释为什么以下分数等同于1/4')
    input_data = example['input']  # Input field (e.g., '4/16')
    output_data = example['output']  # Output field (e.g., explanation for 4/16)

    # Step 2: Construct the final prompt in the required format
    final_prompt = (
        f"<|start_header_id|>user<|end_header_id|>\n\n"
        f"{instruction}\nQuestion: {input_data}<|eot_id|>\n"
        f"<|start_header_id|>assistant<|end_header_id|>\n\nAnswer:"
    )
    tokenized_prompt = tokenizer(final_prompt,add_special_tokens=False)
    tokenized_response = tokenizer(output_data,add_special_tokens=False)
    input_ids = tokenized_prompt["input_ids"]+tokenized_response["input_ids"]
    attention_mask = tokenized_prompt["attention_mask"] + tokenized_response["attention_mask"]
    labels = [-100] * len(tokenized_prompt["input_ids"]) + tokenized_response["input_ids"]


    MAX_LENGTH = 384
    # input_ids, attention_mask, labels = [], [], []
    # instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ",add_special_tokens=False)
    # response = tokenizer(example["output"] ,add_special_tokens=False)
    # input_ids = instruction["input_ids"] + response["input_ids"]+ [tokenizer.eos_token_id]
    # attention_mask = instruction["attention_mask"] + response["attention_mask"]+[1]
    # labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]+ [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [16]:
# tokenizer.eos_token_id
tokenizer.pad_token_id

128004

In [17]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [18]:
tokenized_ds[0]

{'input_ids': [128006,
  882,
  128007,
  271,
  118551,
  113614,
  9554,
  126524,
  46239,
  9174,
  14924,
  25,
  220,
  128009,
  198,
  128006,
  78191,
  128007,
  271,
  16533,
  25,
  88852,
  21043,
  118551,
  113614,
  9554,
  126524,
  46239,
  49543,
  16,
  13,
  111505,
  69978,
  111006,
  108726,
  1811,
  74257,
  36827,
  102210,
  108562,
  40265,
  9554,
  111006,
  114253,
  116051,
  107471,
  65782,
  5486,
  110774,
  65782,
  58291,
  83994,
  126503,
  3922,
  27327,
  113096,
  42399,
  64209,
  104473,
  36651,
  113614,
  3922,
  50285,
  103229,
  120044,
  107079,
  120772,
  91495,
  19361,
  103129,
  35304,
  111689,
  83747,
  33014,
  30358,
  3490,
  17,
  13,
  111020,
  229,
  120383,
  120522,
  102456,
  1811,
  74257,
  36827,
  102456,
  11883,
  17039,
  118882,
  9554,
  107139,
  105,
  108171,
  5486,
  53610,
  28873,
  5486,
  37087,
  104858,
  53953,
  34208,
  121496,
  57942,
  103,
  96412,
  33857,
  103167,
  9554,
  111678,
  

In [19]:
tokenizer.decode(tokenized_ds[0]['input_ids'])

'<|start_header_id|>user<|end_header_id|>\n\n保持健康的三个提示。\nQuestion: <|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\nAnswer:以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'

In [13]:
print(tokenized_ds[0])

{'input_ids': [128006, 882, 128007, 271, 118551, 113614, 9554, 126524, 46239, 9174, 14924, 25, 220, 128009, 198, 128006, 78191, 128007, 271, 16533, 25, 88852, 21043, 118551, 113614, 9554, 126524, 46239, 49543, 16, 13, 111505, 69978, 111006, 108726, 1811, 74257, 36827, 102210, 108562, 40265, 9554, 111006, 114253, 116051, 107471, 65782, 5486, 110774, 65782, 58291, 83994, 126503, 3922, 27327, 113096, 42399, 64209, 104473, 36651, 113614, 3922, 50285, 103229, 120044, 107079, 120772, 91495, 19361, 103129, 35304, 111689, 83747, 33014, 30358, 3490, 17, 13, 111020, 229, 120383, 120522, 102456, 1811, 74257, 36827, 102456, 11883, 17039, 118882, 9554, 107139, 105, 108171, 5486, 53610, 28873, 5486, 37087, 104858, 53953, 34208, 121496, 57942, 103, 96412, 33857, 103167, 9554, 111678, 101828, 103706, 102456, 53953, 3922, 111098, 103048, 45736, 117587, 122603, 121496, 57942, 103, 34208, 117041, 119008, 105610, 118551, 113614, 9554, 120522, 102456, 105369, 33565, 107, 3490, 18, 13, 124022, 94, 120379, 1

## Step4 创建模型

In [20]:
import torch
model = AutoModelForCausalLM.from_pretrained("C:/Pretrained_model/LLM-Research/Llama-3___2-3B-Instruct",low_cpu_mem_usage=True,torch_dtype=torch.half,device_map={'':torch.cuda.current_device()},load_in_8bit=True)
model

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


bin c:\Users\G\miniforge3\envs\transformers\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=

In [10]:
model.dtype

torch.float16

In [12]:
for name,para in model.named_parameters():
    print(name,para.shape,para.dtype)
    # 因为8bit 用于矩阵乘法， 所以全连接层是没有改成int8的

model.embed_tokens.weight torch.Size([128256, 3072]) torch.float16
model.layers.0.self_attn.q_proj.weight torch.Size([3072, 3072]) torch.int8
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 3072]) torch.int8
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 3072]) torch.int8
model.layers.0.self_attn.o_proj.weight torch.Size([3072, 3072]) torch.int8
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 3072]) torch.int8
model.layers.0.mlp.up_proj.weight torch.Size([8192, 3072]) torch.int8
model.layers.0.mlp.down_proj.weight torch.Size([3072, 8192]) torch.int8
model.layers.0.input_layernorm.weight torch.Size([3072]) torch.float16
model.layers.0.post_attention_layernorm.weight torch.Size([3072]) torch.float16
model.layers.1.self_attn.q_proj.weight torch.Size([3072, 3072]) torch.int8
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 3072]) torch.int8
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 3072]) torch.int8
model.layers.1.self_attn.o_proj.weight torc

## LoRA tuning

### LoRA Step1 配置文件

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### PEFT Step2 创建模型

In [22]:
model = get_peft_model(model, config)

In [18]:
for name,para in model.named_parameters():
    print(name,para.dtype)

base_model.model.base_model.model.model.embed_tokens.weight torch.float16
base_model.model.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.int8
base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float32
base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float32
base_model.model.base_model.model.model.layers.0.self_attn.k_proj.weight torch.int8
base_model.model.base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.int8
base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float32
base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float32
base_model.model.base_model.model.model.layers.0.self_attn.o_proj.weight torch.int8
base_model.model.base_model.model.model.layers.0.mlp.gate_proj.weight torch.int8
base_model.model.base_model.model.model.layers.0.mlp.up_proj.weight torch.int8


In [23]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
# model = model.half()

In [24]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear8bitLt(in_features=3072

In [25]:
model.print_trainable_parameters()

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


In [21]:
from torch.utils.data import DataLoader

In [22]:
dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))

In [23]:
ipt = next(enumerate(dl))[1]

In [24]:
ipt

{'input_ids': tensor([[ 35075,     25, 111505,  69978, 113614,   9554, 126524,  46239,   3490,
          72803,     25,    220,  88852,  21043, 118551, 113614,   9554, 126524,
          46239,  49543,     16,     13, 111505,  69978, 111006, 108726,   1811,
          74257,  36827, 102210, 108562,  40265,   9554, 111006, 114253, 116051,
         107471,  65782,   5486, 110774,  65782,  58291,  83994, 126503,   3922,
          27327, 113096,  42399,  64209, 104473,  36651, 113614,   3922,  50285,
         103229, 120044, 107079, 120772,  91495,  19361, 103129,  35304, 111689,
          83747,  33014,  30358,   3490,     17,     13, 111020,    229, 120383,
         120522, 102456,   1811,  74257,  36827, 102456,  11883,  17039, 118882,
           9554, 107139,    105, 108171,   5486,  53610,  28873,   5486,  37087,
         104858,  53953,  34208, 121496,  57942,    103,  96412,  33857, 103167,
           9554, 111678, 101828, 103706, 102456,  53953,   3922, 111098, 103048,
          4573

In [25]:
model(**ipt.to("cuda"))

CausalLMOutputWithPast(loss=tensor(1.5794, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ 0.5571,  1.0928,  3.9297,  ..., -1.9951, -1.9951, -1.9951],
         [-0.3359, -0.1638, -0.0285,  ..., -6.9531, -6.9531, -6.9531],
         [ 1.3213,  2.5742,  0.8296,  ..., -2.0332, -2.0332, -2.0332],
         ...,
         [ 4.6133,  4.9062,  2.1191,  ...,  0.6650,  0.6650,  0.6650],
         [ 2.7559,  2.3516,  2.9180,  ..., -2.4141, -2.4160, -2.4160],
         [-0.8667, -1.5869, -4.8164,  ...,  2.5117,  2.5117,  2.5117]],

        [[ 0.5571,  1.0928,  3.9297,  ..., -1.9951, -1.9951, -1.9951],
         [-0.3359, -0.1638, -0.0285,  ..., -6.9531, -6.9531, -6.9531],
         [ 2.3262,  2.6504,  0.1075,  ..., -1.0654, -1.0645, -1.0645],
         ...,
         [ 2.5469,  4.4570,  2.0723,  ..., -0.5947, -0.5952, -0.5952],
         [ 2.7051,  4.2344,  1.6777,  ..., -0.1199, -0.1201, -0.1203],
         [ 2.7812,  4.2031,  1.7471,  ...,  0.2922,  0.2920,  0.2920]]],
       device='cuda:

## Step5 配置训练参数

In [26]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=3,
    num_train_epochs=1,
    gradient_checkpointing=True,
    # adam_epsilon=1e-4,
)

## Step6 创建训练器

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## Step7 模型训练

In [28]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


{'loss': 1.9765, 'grad_norm': 0.24770013988018036, 'learning_rate': 4.96e-05, 'epoch': 0.01}
{'loss': 2.0277, 'grad_norm': 0.26451408863067627, 'learning_rate': 4.92e-05, 'epoch': 0.02}
{'loss': 1.9526, 'grad_norm': 0.3056473433971405, 'learning_rate': 4.88e-05, 'epoch': 0.02}
{'loss': 1.8342, 'grad_norm': 0.3194566071033478, 'learning_rate': 4.8400000000000004e-05, 'epoch': 0.03}
{'loss': 1.9551, 'grad_norm': 0.6033316254615784, 'learning_rate': 4.8e-05, 'epoch': 0.04}
{'loss': 1.8741, 'grad_norm': 0.4865131676197052, 'learning_rate': 4.76e-05, 'epoch': 0.05}
{'loss': 1.8919, 'grad_norm': 0.46757039427757263, 'learning_rate': 4.72e-05, 'epoch': 0.06}
{'loss': 1.9173, 'grad_norm': 0.5499241948127747, 'learning_rate': 4.6800000000000006e-05, 'epoch': 0.06}
{'loss': 1.9032, 'grad_norm': 0.520094096660614, 'learning_rate': 4.64e-05, 'epoch': 0.07}
{'loss': 1.9409, 'grad_norm': 0.6254320740699768, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.08}
{'loss': 1.9087, 'grad_norm': 0.671566

KeyboardInterrupt: 

In [ ]:
# model = model.base_model

In [32]:
model

LoraModel(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 3072)
          (layers): ModuleList(
            (0): LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=3072, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=3072, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_vector): ModuleDict()
     

In [34]:
lora_parameters = model.get_lora_params()

AttributeError: 'LlamaForCausalLM' object has no attribute 'get_lora_params'

## Step8 模型推理

In [29]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear8bitLt(in_features=3072

In [36]:
ipt = tokenizer("<|start_header_id|>user<|end_header_id|>\n\n{}\nQuestion: {}<|eot_id|>\n".format("你好你是谁？","").strip() + "<|start_header_id|>assistant<|end_header_id|>\n\nAnswer:", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)
# 需要明确指定pad_token_id 是tokenizer的pad_token_id 不然它会自动换成eos_token_id

'user\n\n你好你是谁？\nQuestion: assistant\n\nAnswer: 我是AI助手，叫做LLaMA，我可以帮助回答各种问题，提供信息和建议。如果您需要帮助，请问我可以帮助您什么？'